**Table of contents**<a id='toc0_'></a>    
- [Load Local LoRA](#toc1_1_1_)    
    - [Load LoRA in A1111 way](#toc1_1_2_)    
    - [SFW Example](#toc1_1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### <a id='toc1_1_1_'></a>[Load Local LoRA](#toc0_)

In [ ]:
import torch
from pathlib import Path
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler


pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False,
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.unet.load_attn_procs(lora_model_path)
pipe.load_lora_weights(
    # "/workspace/dso/GenSAR/LoRA/output/DOSRS_v1/512_sd15_lr1e-04/checkpoint-2700",
    "/workspace/dso/GenSAR/LoRA/output/DOSRS_v1/shiprs_512_sd15_lr1e-04/checkpoint-2160",
    weight_name="pytorch_model.bin",
)
pipe.to("cuda")

In [ ]:
from matplotlib import pyplot as plt

user_prompt = "optical remote sensing,aerial view,cargo ship"
# user_nagative_prompt = "aircraft,satellite,incomplete ship"


scales = [0.0, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
num_images_per_scale = 3  # Number of images per scale
fig, axs = plt.subplots(num_images_per_scale, len(scales), figsize=(16, 7))

for i, scale in enumerate(scales):
    images = pipe(
        prompt=user_prompt,
        # negative_prompt=user_nagative_prompt,
        width=512,
        height=512,
        num_inference_steps=25,
        guidance_scale=7.5,
        num_images_per_prompt=num_images_per_scale,
        cross_attention_kwargs={"scale": scale},
        generator=torch.manual_seed(42),
    ).images

    for j in range(num_images_per_scale):
        axs[j, i].imshow(images[j])
        axs[j, i].axis("off")  # To not show axis in each image

    axs[0, i].set_title(
        f"scale: {scale}"
    )  # Setting the title to the scale for the first image in each column

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib import pyplot as plt

# user_prompt = "aerial view, entire small cargo ship, (centered:2), full structure visible, deck, cargo containers, sharp, detailed, high resolution, realistic details"
# user_prompt = "(aerial view of cargo ship:5), tiny cargo ship, elongated target, distinct outline, rectangular shape, centered, multiple cargo decks, deckhouse, bridge structure, onboard cranes, container stacks, Ro-Ro ramps, wide hull"
user_prompt = "remote sensing,aerial,tiny cargo ship in sea,centered bird view,SAR"
user_nagative_prompt = "aircraft,satellite,incomplete ship"

images = pipe(
    prompt=user_prompt,
    negative_prompt=user_nagative_prompt,
    width=512,
    height=512,
    num_inference_steps=25,
    guidance_scale=12,
    num_images_per_prompt=4,
    cross_attention_kwargs={"scale": 0.0},
    generator=torch.manual_seed(42),
).images

n = len(images)  # Total number of images
plt.figure(figsize=(16, 12))  # Size of the complete figure, adjust as necessary

for i in range(n):
    plt.subplot(1, n, i + 1)  # Create subplots: 1 row, n columns, index i+1
    plt.imshow(images[i])
    plt.axis("off")  # To not show axis in each image

plt.show()

### <a id='toc1_1_2_'></a>[Load LoRA in A1111 way](#toc0_)

In [ ]:
import torch

from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Load the base checkpoint
pipeline = StableDiffusionPipeline.from_pretrained(
    "sinkinai/majicMIX-realistic-v5",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False,
).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
    pipeline.scheduler.config, use_karras_sigmas=True
)

In [ ]:
pipeline.load_lora_weights(
    "./data/sd/lora",
    weight_name="BeautyNwsjMajic2-01.safetensors",
)
prompt = "masterpiece, best quality, (realskin:1.5), photograph, Figurative buxom [Nymph:God:5], Metallic hair styled as Braided half-up half-down, at Midday, equirectangular 360, Crystal Cubism, Canon 5d mark 4, Low shutter, photorealistic, realistic,8k,highres,cinemagraph"
negative_prompt = "paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans, extra fingers, fewer fingers, ((watermark:2)), (white letters:1), (multi nipples), bad anatomy, bad hands, text, error, missing fingers, missing arms, missing legs, extra digit, fewer digits, cropped, worst quality, jpeg artifacts, signature, watermark, username, bad feet, Multiple people, blurry, poorly drawn hands, poorly drawn face, mutation, deformed, extra limbs, extra arms, extra legs, malformed limbs, fused fingers, too many fingers, long neck, cross-eyed, mutated hands, polar lowres, bad body, bad proportions, gross proportions, wrong feet bottom render, abdominal stretch, briefs, knickers, kecks, thong, fused fingers, bad body,bad proportion body to legs, wrong toes, extra toes, missing toes, weird toes, 2 body, 2 pussy, 2 upper, 2 lower, 2 head, 3 hand, 3 feet, extra long leg, super long leg, mirrored image, mirrored noise, badhandv4, ng_deepnegative_v1_75t"

images = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=512,
    height=768,
    num_inference_steps=25,
    guidance_scale=12,
    num_images_per_prompt=4,
    cross_attention_kwargs={"scale": 1.0},
    generator=torch.manual_seed(2555690106),
).images

In [ ]:
pipeline.load_lora_weights(
    "./data/sd/lora",
    weight_name="methurlant.safetensors",
)
prompt = "1 beautiful female long hair, druid, forest, methurlant Alan Lee abstract , gradient color realistic, photorealistic fantasy"
negative_prompt = "((worst quality))"

images = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=512,
    height=768,
    num_inference_steps=25,
    guidance_scale=7,
    num_images_per_prompt=4,
    cross_attention_kwargs={"scale": 1.0},
    generator=torch.manual_seed(2555690106),
).images

In [ ]:
# Original w/o LoRA
import matplotlib.pyplot as plt

n = len(images)  # Total number of images
plt.figure(figsize=(16, 12))  # Size of the complete figure, adjust as necessary

for i in range(n):
    plt.subplot(1, n, i + 1)  # Create subplots: 1 row, n columns, index i+1
    plt.imshow(images[i])
    plt.axis("off")  # To not show axis in each image

plt.show()

In [ ]:
# W/ LoRA
import matplotlib.pyplot as plt

n = len(images)  # Total number of images
plt.figure(figsize=(16, 12))  # Size of the complete figure, adjust as necessary

for i in range(n):
    plt.subplot(1, n, i + 1)  # Create subplots: 1 row, n columns, index i+1
    plt.imshow(images[i])
    plt.axis("off")  # To not show axis in each image

plt.show()

### <a id='toc1_1_3_'></a>[SFW Example](#toc0_)

In [ ]:
import torch

from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Load the base checkpoint
pipeline = StableDiffusionPipeline.from_pretrained(
    "philz1337/revanimated",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False,
).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
    pipeline.scheduler.config, use_karras_sigmas=True
)

pipeline.load_lora_weights(
    "samecorner/blindbox", weight_name="blindbox_v1_mix.safetensors"
)
user_prompt = "chinese dragon, furry, fluffy, gradient color ((best quality)), ((masterpiece)), ( extreme detailed, highest detailed, official art, beautiful and aesthetic:1.2),  depth of field, composition, FULL BODY, (CHIBI), (beautiful and detailed eye:1.3),(very happy:1)"
user_negative_prompt = "((badhandv4, easynegative, ng_deepnegative_v1_75t, verybadimagenegative_v1.3)),(bad anatomy), monochrome, grayscale, (text, words, logo, watermark)"

images = pipeline(
    prompt=user_prompt,
    negative_prompt=user_negative_prompt,
    width=512,
    height=768,
    num_inference_steps=25,
    guidance_scale=7,
    num_images_per_prompt=4,
    cross_attention_kwargs={"scale": 0.0},
    generator=torch.manual_seed(42),
).images

# W/ LoRA
import matplotlib.pyplot as plt

n = len(images)  # Total number of images
plt.figure(figsize=(16, 12))  # Size of the complete figure, adjust as necessary

for i in range(n):
    plt.subplot(1, n, i + 1)  # Create subplots: 1 row, n columns, index i+1
    plt.imshow(images[i])
    plt.axis("off")  # To not show axis in each image

plt.show()

In [ ]:
import torch

from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Load the base checkpoint
pipeline = StableDiffusionPipeline.from_pretrained(
    "philz1337/revanimated",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False,
).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
    pipeline.scheduler.config, use_karras_sigmas=True
)

pipeline.load_lora_weights(
    "samecorner/blindbox", weight_name="blindbox_v1_mix.safetensors"
)
user_prompt = "(masterpiece),(best quality),(ultra-detailed), (full body:1.2), 1girl,chibi,cute, smile, white Bob haircut, red eyes, earring, white shirt,black skirt, lace legwear, (sitting on red sofa), seductive posture, smile, A sleek black coffee table sits in front of the sofa and a few decorative items are placed on the shelves, (beautiful detailed face), (beautiful detailed eyes)"
user_negative_prompt = "(low quality:1.3), (worst quality:1.3)"


images = pipeline(
    prompt=user_prompt,
    negative_prompt=user_negative_prompt,
    width=512,
    height=768,
    num_inference_steps=25,
    guidance_scale=7,
    num_images_per_prompt=4,
    cross_attention_kwargs={"scale": 3.0},
    generator=torch.manual_seed(42),
).images

# W/ LoRA
import matplotlib.pyplot as plt

n = len(images)  # Total number of images
plt.figure(figsize=(16, 12))  # Size of the complete figure, adjust as necessary

for i in range(n):
    plt.subplot(1, n, i + 1)  # Create subplots: 1 row, n columns, index i+1
    plt.imshow(images[i])
    plt.axis("off")  # To not show axis in each image

plt.show()

In [ ]:
from matplotlib import pyplot as plt

# pipeline.load_lora_weights(
#     "./data/sd/lora",
#     weight_name="3DMM_V7.safetensors",
# )

pipeline.load_lora_weights(
    "samecorner/blindbox", weight_name="blindbox_v1_mix.safetensors"
)
user_prompt = "(masterpiece),(best quality),(ultra-detailed), (full body:1.2), 1girl,chibi,cute, smile, white Bob haircut, red eyes, earring, white shirt,black skirt, lace legwear, (sitting on red sofa), seductive posture, smile, A sleek black coffee table sits in front of the sofa and a few decorative items are placed on the shelves, (beautiful detailed face), (beautiful detailed eyes)"
user_negative_prompt = "(low quality:1.3), (worst quality:1.3)"

scales = [0.0, 0.5, 1.0, 1.5, 2.0]
num_images_per_scale = 3  # Number of images per scale

fig, axs = plt.subplots(num_images_per_scale, len(scales), figsize=(16, 9))

for i, scale in enumerate(scales):
    images = pipeline(
        prompt=user_prompt,
        negative_prompt=user_negative_prompt,
        width=512,
        height=768,
        num_inference_steps=25,
        guidance_scale=7,
        num_images_per_prompt=num_images_per_scale,
        cross_attention_kwargs={"scale": scale},
        generator=torch.manual_seed(42),
    ).images

    for j in range(num_images_per_scale):
        axs[j, i].imshow(images[j])
        axs[j, i].axis("off")  # To not show axis in each image

    axs[0, i].set_title(
        f"scale: {scale}"
    )  # Setting the title to the scale for the first image in each column

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib import pyplot as plt

# pipeline.load_lora_weights(
#     "./data/sd/lora",
#     weight_name="3DMM_V7.safetensors",
# )

pipeline.load_lora_weights(
    "samecorner/blindbox", weight_name="blindbox_v1_mix.safetensors"
)
user_prompt = "chinese dragon, furry, fluffy, gradient color ((best quality)), ((masterpiece)), ( extreme detailed, highest detailed, official art, beautiful and aesthetic:1.2),  depth of field, composition, FULL BODY, (CHIBI), (beautiful and detailed eye:1.3),(very happy:1)"
user_negative_prompt = "((badhandv4, easynegative, ng_deepnegative_v1_75t, verybadimagenegative_v1.3)),(bad anatomy), monochrome, grayscale, (text, words, logo, watermark)"

scales = [0.0, 0.5, 1.0, 1.5, 2.0]
num_images_per_scale = 3  # Number of images per scale


fig, axs = plt.subplots(num_images_per_scale, len(scales), figsize=(16, 9))

for i, scale in enumerate(scales):
    images = pipeline(
        prompt=user_prompt,
        negative_prompt=user_negative_prompt,
        width=512,
        height=512,
        num_inference_steps=25,
        guidance_scale=7.5,
        num_images_per_prompt=num_images_per_scale,
        cross_attention_kwargs={"scale": scale},
        generator=torch.manual_seed(42),
    ).images

    for j in range(num_images_per_scale):
        axs[j, i].imshow(images[j])
        axs[j, i].axis("off")  # To not show axis in each image

    axs[0, i].set_title(
        f"scale: {scale}"
    )  # Setting the title to the scale for the first image in each column

plt.tight_layout()
plt.show()

In [ ]:
from matplotlib import pyplot as plt

import torch

from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Load the base checkpoint
pipeline = StableDiffusionPipeline.from_pretrained(
    "digiplay/GhostMixV1.2VAE",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False,
).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
    pipeline.scheduler.config, use_karras_sigmas=True
)

# pipeline.load_lora_weights(
#     "./data/sd/lora",
#     weight_name="3DMM_V7.safetensors",
# )

pipeline.load_lora_weights(
    "./data/sd/lora", weight_name="kVoidEnergy-000001.safetensors"
)
user_prompt = "(masterpiece, best quality:1.4),(absurdres, highres, ultra detailed:1.2), (1 handsome man:1.4),(using dark magic:1.4),purple magic,playful illustrations, fractal art,imaginative overlays, artistic fusion,fantastical scenes, evocative narratives, striking visuals, upper body"
user_negative_prompt = "(worst quality, low quality:2), NSFW,monochrome, zombie,overexposure, watermark,text,bad anatomy,bad hand,((extra hands)),extra fingers,too many fingers,fused fingers,bad arm,distorted arm,extra arms,fused arms,extra legs,missing leg,disembodied leg,extra nipples, detached arm, liquid hand,inverted hand,disembodied limb, oversized head,extra body,extra navel,easynegative,(hair between eyes),sketch, duplicate, ugly, huge eyes, text, logo, worst face, (bad and mutated hands:1.3), (blurry:2.0), horror, geometry, bad_prompt, (bad hands), (missing fingers), multiple limbs, bad anatomy, (interlocked fingers:1.2), Ugly Fingers, (extra digit and hands and fingers and legs and arms:1.4), (deformed fingers:1.2), (long fingers:1.2),(bad-artist-anime), bad-artist, bad hand, extra legs ,(ng_deepnegative_v1_75t),((hands on head))"

scales = [0.0, 0.5, 1.0, 1.5, 2.0]
num_images_per_scale = 3  # Number of images per scale

fig, axs = plt.subplots(num_images_per_scale, len(scales), figsize=(16, 9))

for i, scale in enumerate(scales):
    images = pipeline(
        prompt=user_prompt,
        negative_prompt=user_negative_prompt,
        width=512,
        height=512,
        num_inference_steps=25,
        guidance_scale=7.5,
        num_images_per_prompt=num_images_per_scale,
        cross_attention_kwargs={"scale": scale},
        generator=torch.manual_seed(43),
    ).images

    for j in range(num_images_per_scale):
        axs[j, i].imshow(images[j])
        axs[j, i].axis("off")  # To not show axis in each image

    axs[0, i].set_title(
        f"scale: {scale}"
    )  # Setting the title to the scale for the first image in each column

plt.tight_layout()
plt.show()

In [ ]:
import torch
from matplotlib import pyplot as plt
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Load the base checkpoint
# pipeline = StableDiffusionPipeline.from_pretrained(
#     "emilianJR/chilloutmix_NiPrunedFp32Fix",
#     torch_dtype=torch.float32,
#     safety_checker=None,
#     requires_safety_checker=False,
# ).to("cuda")
# pipeline = StableDiffusionPipeline.from_pretrained(
#     "philz1337/revanimated",
#     torch_dtype=torch.float16,
#     safety_checker=None,
#     requires_safety_checker=False,
# ).to("cuda")
pipeline = StableDiffusionPipeline.from_ckpt(
    "https://huggingface.co/hanafuusen2001/ReVAnimated/blob/main/revAnimated_v122.safetensors",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False,
).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
    pipeline.scheduler.config, use_karras_sigmas=True
)

pipeline.load_lora_weights("./data/sd/lora", weight_name="3DMM_V7.safetensors")
# user_prompt = "masterpiece, chinese dragon, long dragon, Loong,fangs,fantasy, mythical, art by( greg rutkowski:0.8), epic lighting, (photo realism:1.2), high quality, highly detailed, masterpiece, epic"
# user_negative_prompt = "(worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers,strange fingers,bad hand (low quality, worst quality:1.4), (bad_prompt:0.8), (monochrome), (greyscale)"
user_prompt = "(upper body), best quality, (masterpiece:1.3),jesus, priest at church, hands together, (looking up:1.1), black hair, blue eyes, (chibi:0.7), streaks of light, cinematic lighting, side lighting, halo, epic composition"
user_negative_prompt = "badhandv4, paintings, sketches, (worst qualit:2), (low quality:2), (normal quality:2), lowers, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, (outdoor:1.6), manboobs, (backlight:1.2), double navel, muted arms, hused arms, neck lace, analog, analog effects, (sunglass:1.4), nipples, nsfw, bad architecture, watermark, (mole:1.5), EasyNegative"


# images = pipeline(
#     prompt=user_prompt,
#     negative_prompt=user_negative_prompt,
#     width=512,
#     height=768,
#     num_inference_steps=30,
#     guidance_scale=7,
#     num_images_per_prompt=4,
#     cross_attention_kwargs={"scale": 2.0},
#     generator=torch.manual_seed(42),
# ).images

# # W/ LoRA
# import matplotlib.pyplot as plt

# n = len(images)  # Total number of images
# plt.figure(figsize=(16, 12))  # Size of the complete figure, adjust as necessary

# for i in range(n):
#     plt.subplot(1, n, i + 1)  # Create subplots: 1 row, n columns, index i+1
#     plt.imshow(images[i])
#     plt.axis("off")  # To not show axis in each image

# plt.show()

scales = [0.0, 0.375, 0.75, 1.125, 1.5]
# scales = [0.0, 1.0]
num_images_per_scale = 3  # Number of images per scale

fig, axs = plt.subplots(num_images_per_scale, len(scales), figsize=(16, 9))

for i, scale in enumerate(scales):
    images = pipeline(
        prompt=user_prompt,
        negative_prompt=user_negative_prompt,
        width=512,
        height=512,
        num_inference_steps=30,
        guidance_scale=7,
        num_images_per_prompt=num_images_per_scale,
        cross_attention_kwargs={"scale": scale},
        generator=torch.manual_seed(42),
    ).images

    for j in range(num_images_per_scale):
        axs[j, i].imshow(images[j])
        axs[j, i].axis("off")  # To not show axis in each image

    axs[0, i].set_title(
        f"scale: {scale}"
    )  # Setting the title to the scale for the first image in each column

plt.tight_layout()
plt.show()

In [ ]:
import torch

from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# Load the base checkpoint
pipeline = StableDiffusionPipeline.from_pretrained(
    "philz1337/revanimated",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False,
).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
    pipeline.scheduler.config, use_karras_sigmas=True
)

pipeline.load_lora_weights(
    "samecorner/blindbox", weight_name="blindbox_v1_mix.safetensors"
)
user_prompt = "(masterpiece),(best quality),(ultra-detailed), (full body:1.2), 1girl,chibi,cute, smile, white Bob haircut, red eyes, earring, white shirt,black skirt, lace legwear, (sitting on red sofa), seductive posture, smile, A sleek black coffee table sits in front of the sofa and a few decorative items are placed on the shelves, (beautiful detailed face), (beautiful detailed eyes)"
user_negative_prompt = "(low quality:1.3), (worst quality:1.3)"


images = pipeline(
    prompt=user_prompt,
    negative_prompt=user_negative_prompt,
    width=512,
    height=768,
    num_inference_steps=25,
    guidance_scale=7,
    num_images_per_prompt=4,
    cross_attention_kwargs={"scale": 3.0},
    generator=torch.manual_seed(42),
).images

# W/ LoRA
import matplotlib.pyplot as plt

n = len(images)  # Total number of images
plt.figure(figsize=(16, 12))  # Size of the complete figure, adjust as necessary

for i in range(n):
    plt.subplot(1, n, i + 1)  # Create subplots: 1 row, n columns, index i+1
    plt.imshow(images[i])
    plt.axis("off")  # To not show axis in each image

plt.show()